<a href="https://colab.research.google.com/github/YolandaMDavis/wildtrack-iqa/blob/additional-tuning/IQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Image Quality Assessment**



Install Required Libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')

!pip install gym==0.17.3 --quiet
!pip install stable-baselines==2.10.2 --quiet
!pip install h5py==3.0.0 --quiet
!pip install tensorflow==1.13.1 --quiet
!pip install tensorboard==1.13.1 --quiet
!pip install Keras==2.2.--quiet
!pip install Keras-Applications==1.0.8 --quiet
!pip install Keras-Preprocessing==1.1.2 --quiet
!pip install keras-rl==0.4.2 --quiet

     |████████████████████████████████| 1.6 MB 5.3 MB/s 
     |████████████████████████████████| 1.0 MB 47.3 MB/s 
     |████████████████████████████████| 240 kB 4.8 MB/s 
     |████████████████████████████████| 4.0 MB 5.1 MB/s 
     |████████████████████████████████| 92.6 MB 1.3 MB/s 
     |████████████████████████████████| 50 kB 7.2 MB/s 
     |████████████████████████████████| 3.2 MB 49.4 MB/s 
     |████████████████████████████████| 367 kB 57.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.7 requires tensorflow>=2.0.0, but you have tensorflow 1.13.1 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement Keras==2.2.--quiet (from versions: 0.2.0, 0.3.0, 0.3.1, 0.3.2, 0.3.3, 1.0.0, 1.0.1, 1.0.2, 1.0.3, 1.0.4, 1.0.5


Copy Model Library and Transformation File to Local Drive


In [2]:
# mount google drive
from google.colab import drive
import shutil

drive.mount('/content/drive') # for google colab. adjust accordingly
PARENT_DIR = '/content/drive/MyDrive/Wildtrack Group/IQA' 


# copy model files
shutil.copy(PARENT_DIR + '/final_model/final_model.zip','final_model.zip')
shutil.copy(PARENT_DIR + '/final_model/transforms.json','transforms.json')

# get downstream/classification task model files
shutil.copy(PARENT_DIR + '/final_model/species_model.pt', 'species_model.pt')
shutil.copy(PARENT_DIR + '/final_model/class_mapping.json', 'class_mapping.json')

Mounted at /content/drive


'class_mapping.json'

Copy and Expand Sample Images

In [3]:
from zipfile import ZipFile

# copy annotated and cropped images imafea
shutil.copy(PARENT_DIR + '/data/Annotated_Cropped_WildTrack.zip', 'Cropped_WildTrack.zip')

# extract zip file
with ZipFile('Cropped_WildTrack.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()

Define Image Conversion functions for Model

In [4]:
from PIL import Image

import albumentations as A
import numpy as np
import torch
import random
import os

transforms = A.load('transforms.json')
data_dir = 'RAW/'

#load pictures into memory
def convert_image(img_path):
    image = Image.open(img_path)
    bands = image.getbands()
    if len(bands) == 1:
      image = image.convert(mode='RGB')
    image = np.array(image)
    image = transforms(image=image)['image']
    return np.array([image])

#Obtain random images for prediction
def select_random_image():
  subdirectories = list(os.walk(data_dir, topdown=False))[:-1]
  while True:
    subdir = random.choice(subdirectories)
    if len(subdir) > 0 and len(subdir[2]) > 0:
      species_rating = subdir[0].rsplit('/', 1)[-1].replace('_', ' ')
      species_class = species_rating.rsplit(' ', 1)[:-1][0]
      image_name = subdir[0] +'/'+ subdir[2][0]
      return image_name, convert_image(image_name), species_class

# retrieve dictionary key given value
def get_key(val, item_dict):
    for key, value in item_dict.items():
        if val == value:
            return key
    return -1
    

Load IQA Model

In [9]:
from stable_baselines import PPO2

# import model for inference
quality_model = PPO2.load('final_model.zip')

Loading a model without an environment, this model cannot be trained until it has a valid environment.


Load Species Classification Model

In [11]:
#Import Species Classification Model
import torch
import json

device = "cpu"
species_model = torch.jit.load('species_model.pt').to(device)

with open('class_mapping.json') as data:
    mappings = json.load(data)

class_mapping = {item['model_idx']: item['class_name'] for item in mappings}


Sample Image Predictions

In [23]:
# Try 10 Random Imaages to assess classification performance
image_predictions = []

for i in range(10):
  image_name, image, species_class = select_random_image()
  image = torch.from_numpy(image).to("cpu")
  prediction = quality_model.predict(image)[0]
  prediction_string = "Usable" if np.rint(np.array(prediction).flatten())[0] > 0 else "Not Usable"
  print('Prediction for image {0}:  {1}\n'.format(image_name, prediction_string))

  if len(species_class.rsplit(' ', 1)) > 1:
      species = species_class.rsplit(' ')[0]
      animal_class = ' '.join(species_class.rsplit(' ')[1:])
  else:
      animal_class = 'Unknown'
      species = species_class

  class_name = species if animal_class == 'Unknown' else species + ': ' + animal_class
  class_idx = get_key(class_name, class_mapping)

  image_predictions.append((image_name, image[0], class_idx, prediction))

Prediction for image RAW/Leopard_African_3/ed818924ce0911ea9d650242ac110002_.jpg:  Usable

Prediction for image RAW/Puma_4/af9fb6cad3dd11ea91590242ac1c0002_.jpg:  Usable

Prediction for image RAW/Tiger_Bengal_3/8e66129e221e11eb9d950242ac110002_.jpg:  Usable

Prediction for image RAW/Jaguar_3/cdeb03f0bfcd11eba8a200155d2c01bc_.jpg:  Usable

Prediction for image RAW/Rhino_White_5/e90c8b04d3dd11ea91590242ac1c0002_.jpg:  Usable

Prediction for image RAW/Tiger_Bengal_3/8e66129e221e11eb9d950242ac110002_.jpg:  Not Usable

Prediction for image RAW/Elephant_African_5/30501b94c0fa11ea82a50242ac1c0002_.jpg:  Usable

Prediction for image RAW/Jaguar_4/c8532248f3c111ea9d950242ac110002_.jpg:  Usable

Prediction for image RAW/Tapir_Lowland_3/725f3d62d3dd11ea91590242ac1c0002_.jpg:  Usable

Prediction for image RAW/Puma_4/af9fb6cad3dd11ea91590242ac1c0002_.jpg:  Usable



Test Image Prediction with Species Classification

In [24]:
# Evaluate images with species classification model

for image_prediction in image_predictions:
  image_name = image_prediction[0]
  image = image_prediction[1]
  y_true = image_prediction[2]
  quality = image_prediction[3]
  obs = image.permute(2, 0, 1).unsqueeze(dim=0).float()
  y_pred = species_model(obs)
  y_pred = y_pred.argmax(dim=1).squeeze().item()

  if y_true == y_pred and quality == 1:
    print('Image {0} was predicted as Usable and could be classified.\n'.format(image_name))
  elif y_true != y_pred and quality == 0:
    print('Image {0} was predicted as Not Usable and could not be classified, as predicted.\n'.format(image_name))
  else:
    print('Quality prediction for Image {0} was not successful\n'.format(image_name))  

Image RAW/Leopard_African_3/ed818924ce0911ea9d650242ac110002_.jpg was predicted as Usable and could be classified.

Image RAW/Puma_4/af9fb6cad3dd11ea91590242ac1c0002_.jpg was predicted as Usable and could be classified.

Image RAW/Tiger_Bengal_3/8e66129e221e11eb9d950242ac110002_.jpg was predicted as Usable and could be classified.

Image RAW/Jaguar_3/cdeb03f0bfcd11eba8a200155d2c01bc_.jpg was predicted as Usable and could be classified.

Image RAW/Rhino_White_5/e90c8b04d3dd11ea91590242ac1c0002_.jpg was predicted as Usable and could be classified.

Quality prediction for Image RAW/Tiger_Bengal_3/8e66129e221e11eb9d950242ac110002_.jpg was not successful

Image RAW/Elephant_African_5/30501b94c0fa11ea82a50242ac1c0002_.jpg was predicted as Usable and could be classified.

Image RAW/Jaguar_4/c8532248f3c111ea9d950242ac110002_.jpg was predicted as Usable and could be classified.

Image RAW/Tapir_Lowland_3/725f3d62d3dd11ea91590242ac1c0002_.jpg was predicted as Usable and could be classified.

Ima